In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import BoundaryNorm

import cartopy.crs as ccrs
from som import Som, cluster

def calc_anom(da):
    climatology_mean = da.groupby("time.month").mean("time")
    climatology_std = da.groupby("time.month").std("time")
    anom = xr.apply_ufunc(
        lambda x, m, s: (x - m) / s,
        da.groupby("time.month"),
        climatology_mean,
        climatology_std,
    )
    return anom

print("Self-Organizing Feature Map Analysis")
# Load data
ds = xr.open_dataset("../Data/u850.nc")
uwnd850 = ds["uwnd"].sel(
    time=slice("1991", "2023"),
    lat=slice(60.0, -10.0),
    lon=slice(100.0, 180.0),
)
uwnd850 = uwnd850.sel(time=uwnd850.time.dt.month.isin([6, 7, 8]))
uwnd850_anom = calc_anom(uwnd850)
uwnd850_anom = uwnd850_anom.drop_vars(["month", "level"])
# Coordinates
lon = uwnd850_anom.lon
lat = uwnd850_anom.lat
nd = len(lon) * len(lat)
nt = uwnd850_anom.time.size

# Reshape to (nd, nt)
x = uwnd850_anom.values.reshape(nt, nd).T

# SOM parameters
m1 = 3
m2 = 3
nter = 5000
alpha0 = 5.0
alphamin = 0.05
taua = 1000
sigma0 = 2.5
taus = 500

# Train SOM
w = Som(x, nd, nt, m1, m2, nter, alpha0, taua, alphamin, sigma0, taus)

# Cluster
cluster1, cluster2, qerr, qerror, count = cluster(x, w, nd, nt, m1, m2)

# ---------------------------
# Average maps per neuron
# ---------------------------
# Pre-allocate storage for average maps (lon x lat x neurons)
avg_maps = np.empty((len(lon), len(lat), m1 * m2), dtype=float)

# Build neuron indices for each sample (1-based like Julia)
neuron_indices = cluster1 + (cluster2 - 1) * m1  # shape (nt,)

for neuron in range(1, m1 * m2 + 1):
    inds = np.where(neuron_indices == neuron)[0]
    if inds.size == 0:
        avg_maps[:, :, neuron - 1] = np.nan
    else:
        # mean over selected time indices
        sel = uwnd850_anom.isel(time=inds).mean(dim="time", keep_attrs=False)
        sel = sel.transpose("lon", "lat")
        avg_maps[:, :, neuron - 1] = sel.values

vmin = -0.05
vmax = 0.05
boundaries = np.linspace(-0.05, 0.05, 11)
norm = BoundaryNorm(boundaries=boundaries, ncolors=256, extend="both")

fig = plt.figure(layout="constrained", figsize=(m1 * 5, m2 * 4))
axes = fig.subplots(3, 3, subplot_kw={"projection": ccrs.PlateCarree()})
for i, ax in enumerate(axes.flat):
    feature = avg_maps[:, :, i]
    im = ax.contourf(
        uwnd850_anom.lon,
        uwnd850_anom.lat,
        feature.T,
        cmap="RdBu_r",
        norm=norm,
        transform=ccrs.PlateCarree(),
    )
    ax.contour(im, colors="black", linewidths=(0.5,))
    ax.set_title(f"Cluster {i + 1}: {count[i]}", fontsize=14)
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    ax.set_extent([100, 180, -10, 60], crs=ccrs.PlateCarree())

cbar = fig.colorbar(
    ScalarMappable(norm=norm, cmap="RdBu_r"),
    ax=axes[:, :],
    location="bottom",
    fraction=0.05,
    pad=0.02,
)
cbar.set_label("U-wind 850hPa Anomaly (m/s)", fontsize=12)
plt.suptitle("U-wind 850hPa Anomaly SOM", fontsize=16)
plt.savefig("../Results/uwnd850_anom_som.png", dpi=300)

# Counting
clusters = cluster1 + (cluster2 - 1) * 3

time = uwnd850_anom.time

df = pd.DataFrame(
    {"cluster": clusters},
    index=pd.to_datetime(time),
)

df["cluster"] = df["cluster"].astype("category")

# Extract year
df["year"] = df.index.to_period(freq="Y")

# Group and count
counts = df.groupby(["cluster", "year"], observed=True).size().reset_index(name="count")
counts = counts.pivot(index="year", columns="cluster", values="count").fillna(0)
counts.to_csv("../Data/u850_cluster_counts.csv")

# Plot time series
# Create one figure with 3x3 subplots
fig, axes = plt.subplots(3, 3, figsize=(15, 12), layout="constrained")
axes = axes.flatten()  # flatten to easily iterate over
t = range(1991, 2024)
for i in range(0, 9):
    ax = axes[i]
    ax.plot(t, counts[i + 1], "-o")
    ax.set_xlabel("Year")
    ax.set_ylabel("Counts")
    ax.set_ylim(-5, 50)
    ax.set_xlim(1990, 2025)
    ax.tick_params(axis="both", tickdir="in")
    ax.grid(True)
    ax.set_title(f"cluster {i + 1}: {int(counts[i + 1].sum())}")

fig.suptitle("Occurences each year")
# Save one combined figure
plt.savefig("../Results/u850a_cluster_timeseries.png")

# u850
# ds = xr.open_dataset("../Data/u850.nc")
ds = ds.sel(time=ds.time.dt.month.isin([6, 7, 8]))
ds.drop_vars("level")
print(ds)

ds_monthly = ds.resample(time="ME").mean()


clima = ds_monthly.groupby("time.month").mean("time")

clima

anom = ds_monthly.groupby("time.month") - clima
print(anom)

import numpy as np
import xarray as xr
from scipy.stats import t

# ---------------------------
# Composite for ANOM_JJA with Significance Test
# ---------------------------

# Ensure data is filtered to JJA and aligned with SOM samples
da_comp = anom  # Assuming 'anom' is your anomaly DataArray
if 'month' in da_comp.coords:
    da_comp = da_comp.drop_vars('month')

# Ensure JJA selection
da_comp_jja = da_comp.sel(time=da_comp.time.dt.month.isin([6, 7, 8]))

# Building neuron labels (1-based) from SOM output
# m1, m2, cluster1, cluster2 come from the SOM training section
neuron_labels = cluster1 + (cluster2 - 1) * m1

# Pre-allocate storage
# avg_maps: Stores the mean anomaly
# sig_maps: Stores the p-value or significance mask
n_lon = len(da_comp_jja.lon)
n_lat = len(da_comp_jja.lat)
n_neurons = m1 * m2

avg_maps_comp = np.empty((n_lon, n_lat, n_neurons), dtype=float)
sig_maps_comp = np.empty((n_lon, n_lat, n_neurons), dtype=float)

print("Calculating composites and significance...")

for neuron in range(1, n_neurons + 1):
    inds = np.where(neuron_labels == neuron)[0]
    
    if inds.size < 2: # Need at least 2 samples for variance
        avg_maps_comp[:, :, neuron - 1] = np.nan
        sig_maps_comp[:, :, neuron - 1] = np.nan
    else:
        # Select the subset of data for this cluster
        subset = da_comp_jja.isel(time=inds)
        
        # 1. Calculate Mean
        # keep_attrs=False prevents xarray from dropping coords sometimes
        clus_mean = subset.mean(dim="time", skipna=True)
        
        # 2. Calculate Std Dev (ddof=1 for sample std)
        clus_std = subset.std(dim="time", ddof=1, skipna=True)
        
        # 3. Calculate N (sample size)
        # If there are NaNs in space, count valid observations per grid point
        n_samples = subset.count(dim="time") 
        
        # 4. T-statistic: t = (mean - 0) / (std / sqrt(n))
        # We assume population mean of anomalies is 0
        se = clus_std / np.sqrt(n_samples)
        t_stat = clus_mean / se
        
        # 5. Two-tailed P-value
        # degrees of freedom = n - 1
        dof = n_samples - 1
        pval = 2 * t.sf(np.abs(t_stat), dof)
        
        # Transpose to (lon, lat)
        if 'lon' in clus_mean.dims and 'lat' in clus_mean.dims:
            clus_mean = clus_mean.transpose("lon", "lat")
            pval = pval.transpose("lon", "lat") # pval is an xarray object here
            
        avg_maps_comp[:, :, neuron - 1] = clus_mean.values
        sig_maps_comp[:, :, neuron - 1] = pval.values

# Update global variables
avg_maps = avg_maps_comp
# We will use this for plotting (values < 0.05 are significant)
p_values_maps = sig_maps_comp

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.cm import ScalarMappable
from matplotlib.colors import BoundaryNorm

# ---------------------------
# Visualization with Stippling
# ---------------------------

# Plot parameters
vmin = -0.5 # Adjust based on your data range
vmax = 0.5
boundaries = np.linspace(vmin, vmax, 21)
norm = BoundaryNorm(boundaries=boundaries, ncolors=256, extend="both")

# Significance Level (e.g., 95%)
sig_level = 0.05 

fig = plt.figure(layout="constrained", figsize=(m1 * 5, m2 * 4))
axes = fig.subplots(
    m2, m1, subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)}
)

for i, ax in enumerate(axes.flat):
    # Data for this cluster
    feature = avg_maps[:, :, i]
    pval = p_values_maps[:, :, i]
    
    # 1. Plot the color shading (Mean Anomaly)
    im = ax.contourf(
        da_comp_jja.lon,
        da_comp_jja.lat,
        feature.T, # Transpose if necessary based on your array shape
        cmap="RdBu_r",
        norm=norm,
        transform=ccrs.PlateCarree(),
    )
    
    # 2. Add Contours for clarity
    # ax.contour(im, colors="k", linewidths=(0.5,), transform=ccrs.PlateCarree())
    
    # 3. Add Stippling for Significance
    # We mask values where p-value > 0.05 (insignificant) so they are NOT hatched
    # or we plot specific levels.
    
    # Method: Contourf with hatches
    # levels: [0, sig_level, 1] -> 
    # Bin 1: 0 to 0.05 (Significant) -> Apply Hatch
    # Bin 2: 0.05 to 1 (Not Significant) -> No Hatch
    ax.contourf(
        da_comp_jja.lon,
        da_comp_jja.lat,
        pval.T,
        levels=[0, sig_level, 1],
        hatches=['...', ''], # '...' = dots, '' = nothing
        colors='none', # Make the fill transparent, only show hatches
        transform=ccrs.PlateCarree()
    )

    # Titles and formatting
    ax.set_title(f"Cluster {i + 1} (n={count[i]})", fontsize=14)
    ax.coastlines()
    
    # Extent
    ax.set_extent([0, 180, -60, 60], crs=ccrs.PlateCarree())
    
    # Gridlines
    gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False

# Colorbar
cbar = fig.colorbar(
    ScalarMappable(norm=norm, cmap="RdBu_r"),
    ax=axes[:, :],
    location="bottom",
    fraction=0.05,
    pad=0.02,
)
cbar.set_label("U-wind 850hPa Anomaly (m/s) [Stippling: p < 0.05]", fontsize=12)
plt.suptitle("U-wind 850hPa Anomaly Composite (JJA)", fontsize=16)

save_path = "../Results/uwnd850_anom_som_significant.png"
plt.savefig(save_path, dpi=300)
print(f"Figure saved to {save_path}")
plt.show()